In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from aircraft_dataset import aircraft_dataset_split
from aircraft_dataset import AircraftDatasetFoldIterator

import sys
sys.path.append('../')
import util.reports as rp

In [ ]:
# Loading dataset
dataset = pd.read_csv('exports/2019-07-25 16-26-22/segment_1.csv')
predictors = dataset.iloc[:,:680].values
responses = dataset.iloc[:,680].values
measurements = dataset.iloc[:,681].values
np.unique(responses)

In [ ]:
# Splits into training and holdout set
X,y,X_holdout,y_holdout,measurements,measurement_holdout = aircraft_dataset_split(predictors,responses,measurements,
                                                                                  return_measurements=True)
lc = LabelEncoder()
lc.fit(y)
y = lc.transform(y)
y_holdout = lc.transform(y_holdout)
iterator = AircraftDatasetFoldIterator(X,y,measurements,folds=3)
iterator.build()

In [ ]:
# Performs grid search
sc = StandardScaler()
clf = MLPClassifier(solver='lbfgs', max_iter=200)
estimators = [('normalizer', sc), ('classifier', clf)]
pipe = Pipeline(estimators)

architecture_choices = [(100,30), (100,), (50,)]
alpha_choices = [0,0.01,0.03,0.1,0.5,1,2]
activation_choices = ['logistic', 'relu']

hyperparams = [{
    'classifier__hidden_layer_sizes': architecture_choices,
    'classifier__alpha': alpha_choices,
    'classifier__activation': activation_choices
}]

validator = GridSearchCV(pipe, cv=iterator, param_grid=hyperparams, scoring='accuracy', n_jobs=-1,verbose = 1,iid = False)
validator.fit(X,y)
rp.report_grid_search(validator.cv_results_)

In [ ]:
# Training set performance
y_pred = validator.predict(X)
rp.report_classification(y,y_pred,avg='macro',title='Training')

In [ ]:
# Holdout set performance
y_pred = validator.predict(X_holdout)
rp.report_classification(y_holdout,y_pred,avg='macro',title='Holdout')